## Normalization

In [13]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [14]:
X_processed = pd.read_csv('saved/feature_engineered_data.csv')
X_processed_copy = X_processed.copy()

In [15]:
numerical_cols = X_processed.columns

X_scaled = StandardScaler().fit_transform(X_processed)
X_scaled = pd.DataFrame(X_scaled, columns=X_processed.columns)

In [16]:
# inference_preprocessor = ColumnTransformer(
#     transformers=[
#         # ('num', numerical_pipeline, numerical_cols),
#         ('cat', numerical_pipeline, [i for i in numerical_cols if i != "target_default"])
#     ],
#     # remainder='drop'  # <--- IMPORTANT: Drop the remaining columns
# )

# inference_preprocessor.fit(X_processed_copy.drop(columns=["target_default"]))

# import pickle

# # Save label encoders to disk
# with open('saved/inference_preprocessor.pkl', 'wb') as f:
#     pickle.dump(inference_preprocessor, f)

In [17]:
# from sklearn.feature_selection import f_classif

# X_train_rus, y_train_rus = X_processed.drop(columns=['target_default']), X_processed['target_default']

# f_scores, p_values = f_classif(X_train_rus, y_train_rus)

# # Create a DataFrame to display feature importance
# yo = pd.DataFrame({'Feature': X_train_rus.columns, 'F-Score': f_scores, 'P-Value': p_values})

# # Sort by F-Score (higher F-Score means more important)
# feature_importance = yo.sort_values(by='F-Score', ascending=False)

# # Print the feature importance
# print(feature_importance)

## Clustering

In [18]:
from sklearn.cluster import KMeans


# no_fb = X_processed.drop(columns=["facebook_profile"])
# Perform K-Means clustering on normalized data
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to the original dataset
X_processed['Cluster'] = clusters
# X_processed=X_processed.drop(columns=['target_default'])
print(pd.Series(clusters).value_counts().sort_index())
# Display the clustered data
X_processed.to_csv('saved/clustered_data.csv', index = False)

(X_processed.head())

0     1634
1    10382
2    13962
3    12495
4     3268
Name: count, dtype: int64


,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,...,shipping_state_avg_facebook_profile,score_1_div_score_2,score_1_minus_score_2,state_x_real_state,state_x_shipping_state,state_real_state_avg_score_1,state_shipping_state_avg_score_2,score_1_sq,score_2_sq,Cluster
0,0,0,10,350.0,101.800832,0.259555,108.427273,0.40,10.128027,3.610918,...,0.324643,0.000000,-10,44,528,1.896319,17.306748,0,100,0
1,0,3,16,370.0,97.062615,0.942655,92.002546,0.24,0.000000,0.000000,...,0.322504,0.187500,-13,47,940,1.859049,17.294872,9,256,2
2,1,3,9,360.0,100.027073,0.351918,112.892453,0.29,8.883074,3.610918,...,0.361389,0.333333,-6,96,608,1.906542,16.590909,9,81,3
3,0,0,21,510.0,101.599485,0.987673,94.902491,0.32,0.000000,0.000000,...,0.333609,0.000000,-21,69,391,1.915888,17.650000,0,441,1
4,0,2,1,500.0,98.474289,0.532539,118.126207,0.18,0.000000,0.000000,...,0.324643,2.000000,1,141,564,1.947590,17.600000,4,1,2
